https://keras.io/api/applications/# Pawpularity Contest

Submissions are scored on the root mean squared error **RMSE**.

Guides to use:
*   Good Paper ==> https://dl.acm.org/doi/pdf/10.1145/3209693.3209698
*   Transfer Learning ==> https://keras.io/api/applications/

Things to do in order to increase efficiency:
1.  Add more tags to the dataset by using a pretrained model of classification
2.  Try common techniques for dealing with imbalanced data like:
  *  Class weighting
  *  Oversampling
3.  Try different Learning Rates and Optimizers


# Import Libraries

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input, Concatenate
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import MeanSquaredError, MeanSquaredLogarithmicError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import math

In [ ]:
# Use both CPUs
# inter_op_parallelism_threads = 2;

# See if GPU is used
tensorflow.test.gpu_device_name()

In [3]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime, os

In [4]:
pawpularity_path = '../input/petfinder-pawpularity-score';
working_path = '../working';
inception_weights_path = '../input/inceptionresnetv2/inceptionresnetv2weightstfdimnotop.h5';

# Read Dataframe and split to train & validation

In [5]:
df = pd.read_csv(pawpularity_path+'/train.csv')
df.head()

In [6]:
df['Id'] = df['Id'] + '.jpg';
df['Id']

In [7]:
train_df, val_df = train_test_split(df,test_size=0.001)

In [8]:
train_df.count()

In [9]:
val_df.count()

In [10]:
fig = plt.figure(figsize = (15,10));
ax = fig.gca();
df['Pawpularity'].hist(ax = ax);

## As we can see the data are imbalanced. We must do something about it later.

In [11]:
plt.figure(figsize=(15,10));
ax = sns.heatmap(df.corr());

## As we can see none of metadata is so important. So we don't use them for training.

## Show 9 random images

In [13]:
import random;
rows, cols = 3, 3;
fig, axs = plt.subplots(rows, cols, figsize=(15,15));
fig.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.2, wspace=0.4);
for i in range(rows):
    for j in range(cols):
      random_image = random.randint(0,len(df)-1);
      img = mpimg.imread(pawpularity_path+'/train/'+df['Id'][random_image]);
      axs[i,j].imshow(img);
      axs[i,j].axis('off');
      axs[i,j].set_title(f'Pawpularity: {df["Pawpularity"][random_image]}',{'fontsize': 20});

In [14]:
# tabular_columns = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

# def preprocess(image, tabular):
#     image_string = tensorflow.io.read_file(pawpularity_path+'/train/'+image);
#     image = tensorflow.image.decode_jpeg(image_string, channels=3);
#     image = tensorflow.cast(image, tensorflow.float32) / 255.0;
#     image = tensorflow.image.central_crop(image, 1.0);
#     image = tensorflow.image.resize(image, (300, 300));
#     return (image, tabular[0:12]), tabular[12]

In [15]:
# train_images = train_df['Id'];
# val_images = val_df['Id'];
# rest_of_train_data = train_df.drop('Id',axis=1);
# rest_of_val_data = val_df.drop('Id',axis=1);

# Create Neural Network

In [16]:
InceptionResNetV2 = InceptionResNetV2(
    weights=inception_weights_path,
    include_top=False,
    input_tensor=None,
    input_shape=(300,300,3),
    pooling='max'
)

In [17]:
# train = tensorflow.data.Dataset.from_tensor_slices((train_images, rest_of_train_data)).map(preprocess).shuffle(216).batch(62).prefetch(2)
# validation = tensorflow.data.Dataset.from_tensor_slices((val_images, rest_of_val_data)).map(preprocess).batch(10).prefetch(2)

train_data_generator = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.2,
    rescale = 1.0/255.0
);

val_data_generator = ImageDataGenerator(
    rescale = 1.0/255.0
);


train = train_data_generator.flow_from_dataframe(
    dataframe=train_df,
    directory=pawpularity_path+"/train/",
    x_col="Id",
    y_col="Pawpularity",
    batch_size=64,
    shuffle=True,
    class_mode="raw",
    target_size=(300,300)
);

validation = val_data_generator.flow_from_dataframe(
    dataframe=val_df,
    directory=pawpularity_path+"/train/",
    x_col="Id",
    y_col="Pawpularity",
    batch_size=10,
    shuffle=False,
    class_mode="raw",
    target_size=(300,300)
);

In [28]:
image_input = Input(shape=(300, 300, 3))
# tabular_input = Input(len(tabular_columns))

image_x = InceptionResNetV2(image_input)
image_x = Dropout(0.1)(image_x)
image_x = Flatten()(image_x)
# image_x = Dense(64, activation="relu", kernel_regularizer=tensorflow.keras.regularizers.l2())(image_x)

# tabular_x = Dense(16, activation="relu")(tabular_input)
# tabular_x = Dense(16, activation="relu")(tabular_x)
# tabular_x = Dense(16, activation="relu")(tabular_x)
# tabular_x = Dense(16, activation="relu", kernel_regularizer=tensorflow.keras.regularizers.l2())(tabular_x)

# x = Concatenate(axis=1)([image_x, tabular_x])
x = Dense(10, activation="relu")(image_x)
# x = Dropout(0.1)(x)
output = Dense(1, activation="linear")(x)

model = Model(inputs=image_input,outputs=output)
model.summary()

Despite the widespread popularity of Adam, recent research papers have noted that it can fail to converge to an optimal solution under specific settings. The paper Improving Generalization Performance by Switching from Adam to SGD demonstrates that adaptive optimization techniques such as Adam generalize poorly compared to SGD.

In [29]:
# Settings
adam_lr =  0.001#@param {type:"slider", min:0.0001, max:0.01, step:0.0001}
sgd_lr =  0.01#@param {type:"slider", min:0.001, max:0.09, step:0.001}

In [30]:
# tf.keras.optimizers.Adam(
#     learning_rate=0.0003, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
#     name='Adam', **kwargs
# )

# tf.keras.optimizers.SGD(
#     learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD", **kwargs
# )

# Lets tweak learning rate to see rate of conversion
adam = Adam(learning_rate = adam_lr);
sgd = SGD(learning_rate = sgd_lr);
mse_loss = MeanSquaredError();
msle_loss  = MeanSquaredLogarithmicError (); #For SGD
rmse = RootMeanSquaredError(name='rmse');

reduce_lr = ReduceLROnPlateau(monitor='rmse', patience=3, verbose=1, factor=0.25, min_lr=0.00001);

early_stop = EarlyStopping(
    monitor="rmse",
    min_delta=0.01,
    patience=10,
    verbose=1,
    mode="min",
    baseline=None,
    restore_best_weights=True,
);

def scheduler(epoch, learning_rate):
    if epoch < 2:
        new_learning_rate = learning_rate
    else:  
        new_learning_rate = learning_rate * tensorflow.math.exp(-0.1);
    print(f'Learning rate = {new_learning_rate:.6f}');
    return new_learning_rate;

learning_scheduler = LearningRateScheduler(scheduler);

model.compile(loss=mse_loss, optimizer=adam, metrics=['mae',rmse]);
callbacks=[learning_scheduler, early_stop, reduce_lr];

# Train Model

In [31]:
epochs =  40#@param {type:"slider", min:10, max:300, step:10}
history = model.fit(train, validation_data=validation, epochs=epochs, verbose=1, callbacks=callbacks)

In [32]:
plt.plot(history.history['rmse'])
plt.plot(history.history['val_rmse'])
plt.ylabel('RMSE')
plt.xlabel('Epoch')
plt.legend(['Train_RMSE','Validation_RMSE'])
plt.show()

# See Results

In [56]:
val_predictions = model.predict(validation).reshape(-1);
images, scores = validation.next()
k = 0

rows, cols = 5, 2;
fig, axs = plt.subplots(rows, cols, figsize=(15,15));
fig.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.2, wspace=0.4);
for i in range(rows):
    for j in range(cols):
        axs[i,j].imshow(images[k]);
        axs[i,j].axis('off');
        axs[i,j].set_title(f'Real:{scores[k]} - Predicted:{val_predictions[k]:.2f}',{'fontsize': 20});
        k += 1

# Create submissions csv

For each Id in the test set, you must predict a probability for the target variable, Pawpularity. The file should contain a header and have the following format:

Id, Pawpularity \
0008dbfb52aa1dc6ee51ee02adf13537, 99.24 \
0014a7b528f1682f0cf3b73a991c17a0, 61.71 \
0019c1388dfcd30ac8b112fb4250c251, 6.23 \
00307b779c82716b240a24f028b0031b, 9.43 \
00320c6dd5b4223c62a9670110d47911, 70.89 \
etc.

In [ ]:
test_df = pd.read_csv(pawpularity_path+'/test.csv')
test_df.head()

In [ ]:
test_df['file_path'] = test_df['Id'] + '.jpg';
test_df.head()

In [ ]:
def test_preprocess(image, tabular):
    image_string = tensorflow.io.read_file(pawpularity_path+'/test/'+image);
    image = tensorflow.image.decode_jpeg(image_string, channels=3);
    image = tensorflow.cast(image, tensorflow.float32) / 255.0;
    image = tensorflow.image.central_crop(image, 1.0);
    image = tensorflow.image.resize(image, (300, 300));
    return (image, tabular), 0

test_images = test_df['file_path'];
rest_of_test_data = test_df.drop('Id',axis=1);
rest_of_test_data = rest_of_test_data.drop('file_path',axis=1);
rest_of_test_data.head()

In [ ]:
test = tensorflow.data.Dataset.from_tensor_slices((test_images, rest_of_test_data)).map(test_preprocess).batch(8).prefetch(2)

In [ ]:
predicted_scores = model.predict(test).reshape(-1);
predicted_scores

In [ ]:
test_df['Pawpularity'] = predicted_scores;
submission_df = test_df.reindex(['Id','Pawpularity'],axis=1);
submission_df

In [ ]:
submission_df.to_csv(working_path+'/submission.csv', index=False);